In [11]:
import os
import json
import logging
import asyncio
import nest_asyncio  # Important for running asyncio in Jupyter notebooks
from typing import Dict, List, Any, Optional, Tuple
from datetime import datetime

# LlamaIndex imports
from llama_index.core import Settings, VectorStoreIndex, Document, StorageContext, load_index_from_storage

from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.workflow import Context, InputRequiredEvent, HumanResponseEvent
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import SentenceSplitter

In [12]:
# Import the component notebooks
%run "./assessment_criteria.ipynb"
%run "./guidelines_helper.ipynb"
%run "./assessment_tools.ipynb"

Assessment criteria loaded successfully.
Guidelines retriever loaded successfully.
Assessment criteria loaded successfully.
Guidelines retriever loaded successfully.
Assessment tools loaded successfully.


In [13]:

# Apply nest_asyncio to allow asyncio to work in Jupyter notebooks
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("InsuranceAgentSystem")
logger.setLevel(logging.DEBUG)

# Environment variables - in production, load these from environment
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY', 'your-api-key')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT', 'your-endpoint')
AZURE_GPT_API_VERSION = os.getenv('AZURE_GPT_API_VERSION', '2025-01-01-preview')
AZURE_EMBEDDING_API_VERSION = os.getenv('AZURE_EMBEDDING_API_VERSION', '2023-12-01-preview')

class InsuranceAgentSystem:
    """Main class for the Insurance Agent System implementing Stage 4 (Rank or Reject)"""
    
    def __init__(self):
        """Initialize the system with LLM, embeddings and guidelines"""
        self.setup_llm_and_embeddings()
        self.setup_guidelines_vector_store()
        self.create_agents()
        
    def setup_llm_and_embeddings(self):
        """Configure the LLM and embedding models"""
        self.llm = AzureOpenAI(
            model="gpt-4o",
            deployment_name="gpt-4o",
            api_key=AZURE_OPENAI_API_KEY,
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_version=AZURE_GPT_API_VERSION,
            temperature=0.1,  # Lower temperature for consistent decisions
            timeout=60,
        )
        
        self.embed_model = AzureOpenAIEmbedding(
            model="text-embedding-ada-002",
            deployment_name="text-embedding-ada-002",
            api_key=AZURE_OPENAI_API_KEY,
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_version=AZURE_EMBEDDING_API_VERSION,
        )
        
        # Set as default models
        Settings.llm = self.llm
        Settings.embed_model = self.embed_model
        
        logger.info("LLM and embedding models initialized")
    
    def setup_guidelines_vector_store(self):
        """Initialize the guidelines vector store from the PDF"""
        try:
            # Define paths
            pdf_path = "data/UW Commercial Insurance Manual.pdf"
            persist_dir = "storage/guidelines_index"
            
            # Check if index already exists
            if os.path.exists(persist_dir) and os.listdir(persist_dir):
                logger.info(f"Loading existing index from {persist_dir}")
                storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
                self.guidelines_index = load_index_from_storage(storage_context, index_cls=VectorStoreIndex)
            else:
                # Load and process PDF if needed
                logger.info(f"Loading PDF document from {pdf_path}")
                pdf_reader = PDFReader()
                documents = pdf_reader.load_data(pdf_path)
                logger.info(f"Loaded {len(documents)} documents from PDF")
                
                # Configure settings with node parser
                Settings.node_parser = SentenceSplitter(
                    chunk_size=1024,
                    chunk_overlap=200,
                    separator=" ",
                    paragraph_separator="\n\n",
                )
                
                # Create index
                logger.info("Creating vector store index...")
                self.guidelines_index = VectorStoreIndex.from_documents(documents)
                logger.info("Successfully created index")
                
                # Persist the index
                os.makedirs(persist_dir, exist_ok=True)
                logger.info(f"Persisting index to {persist_dir}")
                self.guidelines_index.storage_context.persist(persist_dir=persist_dir)
                logger.info("Successfully persisted index")
            
            # Create retriever
            self.guidelines_retriever = self.guidelines_index.as_retriever(similarity_top_k=3)

            # Initialize the specialized retrievers and assessment tools
            self.guidelines_helper = GuidelinesRetriever(self.guidelines_retriever)
            self.assessment_tools = AssessmentTools(self.guidelines_helper)
            
            logger.info("Guidelines vector store and assessment tools initialized successfully")
        except Exception as e:
            logger.error(f"Error setting up guidelines vector store: {str(e)}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
            self.guidelines_index = None
            self.guidelines_retriever = None
            self.guidelines_helper = None
            self.assessment_tools = None
            logger.warning("Using mock guidelines data as fallback")
    
    def create_agents(self):
        """Create all the specialized agents for the workflow"""
        # The actual agent creation happens at the end of this method to keep all tools in scope
        # Create all the agents with their tools
        
               # Hazard Agent Tools
        async def evaluate_hazard_classification(ctx: Context, property_data: Dict[str, Any]) -> str:
            """
            Evaluates the hazard classification of a property based on building type, 
            construction materials, and occupancy.
            
            Args:
                property_data: Dictionary containing property details
                
            Returns:
                A description of the hazard assessment with score (1-5)
            """
            logger.info("Starting hazard classification assessment")
            current_state = await ctx.get("state")
            
            try:
                # Use the assessment tools to evaluate hazard
                if hasattr(self, 'assessment_tools') and self.assessment_tools:
                    assessment_result = await self.assessment_tools.evaluate_hazard(property_data)
                    hazard_score = assessment_result.get("score", 3.0)
                else:
                    # Fallback to original logic if assessment tools aren't available
                    building_type = property_data.get("building_type", "")
                    construction = property_data.get("construction", "")
                    occupancy = property_data.get("occupancy", "")
                    
                    logger.info(f"Analyzing property: {building_type} building with {construction} construction")
                    
                    # Sample logic - would be more sophisticated in production
                    hazard_factors = {
                        "building_type": {
                            "Office": 1,
                            "Retail": 2,
                            "Manufacturing": 3,
                            "Warehouse": 3,
                            "Heavy Industrial": 4,
                            "Nightclub": 5
                        },
                        "construction": {
                            "Concrete": 1,
                            "Steel Frame": 2,
                            "Brick": 2,
                            "Wood Frame": 4,
                            "Mixed": 3
                        }
                    }

                    bt_score = hazard_factors["building_type"].get(building_type, 3)
                    const_score = 3  # Default score
                    for material, score in hazard_factors["construction"].items():
                        if material.lower() in construction.lower():
                            const_score = score
                            break

                    # Calculate overall hazard score
                    hazard_score = (bt_score + const_score) / 2
                    
                    assessment_result = {
                        "score": hazard_score,
                        "building_type_assessment": f"{building_type}: Risk level {bt_score}",
                        "construction_assessment": f"{construction}: Risk level {const_score}",
                        "occupancy_details": occupancy
                    }
                
                logger.info(f"Calculated hazard score: {hazard_score}")
                
                # Store the result in context
                if "assessment_results" not in current_state:
                    current_state["assessment_results"] = {}
                
                current_state["assessment_results"]["hazard"] = assessment_result
                
                # Track completion as a list
                if "completed_assessments" not in current_state:
                    current_state["completed_assessments"] = []
                    
                if "hazard" not in current_state["completed_assessments"]:
                    current_state["completed_assessments"].append("hazard")
                
                logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
                await ctx.set("state", current_state)
                
                logger.info("Hazard assessment completed and stored in context")
                return f"Hazard assessment completed. Overall hazard score: {hazard_score:.1f}/5.0"
            except Exception as e:
                logger.error(f"Error in hazard classification: {str(e)}")
                return f"Error in hazard classification: {str(e)}"
        
        async def query_hazard_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for hazard-related information.
            
            Args:
                query: The search query for hazard-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying hazard guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    building_type = ""
                    construction = ""
                    
                    # Try to extract building type and construction from query
                    if "building" in query.lower() and "with" in query.lower():
                        parts = query.lower().split("with")
                        if len(parts) >= 2:
                            building_parts = parts[0].split("for")
                            if len(building_parts) >= 2:
                                building_type = building_parts[1].strip()
                            construction = parts[1].strip()
                    
                    # Use enhanced guideline retrieval
                    guidelines_info = await self.guidelines_helper.get_hazard_guidelines(
                        building_type or query, 
                        construction or "general"
                    )
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "\n\n".join([node.text for node in retrieval_results])
                
                # # Store in context
                # current_state = await ctx.get("state")
                # if "guidelines_retrieved" not in current_state:
                #     current_state["guidelines_retrieved"] = {}
                
                # current_state["guidelines_retrieved"]["hazard"] = guidelines_info
                # await ctx.set("state", current_state)
                
                # NEW: Log actual guidelines content
                logger.debug(f"Retrieved hazard guidelines:\n{guidelines_info}")

                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["hazard"] = guidelines_info
                await ctx.set("state", current_state)

                
                logger.info("Successfully retrieved hazard guidelines")
                return f"Retrieved hazard guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying hazard guidelines: {str(e)}")
                return "Error retrieving hazard guidelines. Using default assessment only."
        
        # Vulnerability Agent Tools
        async def evaluate_vulnerability(ctx: Context, security_data: Dict[str, Any]) -> str:
            """
            Evaluates the vulnerability of a property based on security systems,
            protective measures, and other safety features.
            
            Args:
                security_data: Dictionary containing security details
                
            Returns:
                A description of the vulnerability assessment with score (1-5)
            """
            logger.info("Starting vulnerability assessment")
            current_state = await ctx.get("state")
            
            try:
                # Use the assessment tools to evaluate vulnerability
                if hasattr(self, 'assessment_tools') and self.assessment_tools:
                    assessment_result = await self.assessment_tools.evaluate_vulnerability(security_data)
                    vulnerability_score = assessment_result.get("score", 3.0)
                else:
                    # Fallback to original logic if assessment tools aren't available
                    has_sprinklers = security_data.get("sprinklers", False)
                    alarm_system = security_data.get("alarm_system", "None")
                    
                    logger.info(f"Security features: Sprinklers={has_sprinklers}, Alarm={alarm_system}")
                    
                    # Basic scoring
                    sprinkler_score = 1 if has_sprinklers else 4
                    
                    alarm_scores = {
                        "None": 5,
                        "Local": 3,
                        "Monitored": 2,
                        "Grade A - 24hr Monitored": 1
                    }
                    alarm_score = alarm_scores.get(alarm_system, 3)
                    
                    # Calculate overall vulnerability score
                    vulnerability_score = (sprinkler_score + alarm_score) / 2
                    
                    assessment_result = {
                        "score": vulnerability_score,
                        "sprinkler_assessment": f"Sprinklers: {'Present' if has_sprinklers else 'Absent'}, Risk level {sprinkler_score}",
                        "alarm_assessment": f"Alarm: {alarm_system}, Risk level {alarm_score}"
                    }
                
                logger.info(f"Calculated vulnerability score: {vulnerability_score}")

                # Store the result
                if "assessment_results" not in current_state:
                    current_state["assessment_results"] = {}
                
                current_state["assessment_results"]["vulnerability"] = assessment_result
                
                # Track completion as a list
                if "completed_assessments" not in current_state:
                    current_state["completed_assessments"] = []
                    
                if "vulnerability" not in current_state["completed_assessments"]:
                    current_state["completed_assessments"].append("vulnerability")
                
                logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
                await ctx.set("state", current_state)
                
                logger.info("Vulnerability assessment completed and stored in context")
                return f"Vulnerability assessment completed. Overall vulnerability score: {vulnerability_score:.1f}/5.0"
            except Exception as e:
                logger.error(f"Error in vulnerability assessment: {str(e)}")
                return f"Error in vulnerability assessment: {str(e)}"
        
        async def query_vulnerability_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for vulnerability-related information.
            
            Args:
                query: The search query for vulnerability-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying vulnerability guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    building_type = ""
                    building_age = 0
                    
                    # Try to extract building type and age from query
                    if "for" in query.lower() and "buildings" in query.lower():
                        parts = query.lower().split("for")
                        if len(parts) >= 2:
                            building_parts = parts[1].split("buildings")
                            if len(building_parts) >= 1:
                                building_type = building_parts[0].strip()
                    
                    # See if there's age info
                    if "years" in query.lower():
                        parts = query.lower().split("years")
                        if len(parts) >= 1:
                            for word in parts[0].split()[::-1]:
                                if word.isdigit():
                                    building_age = int(word)
                                    break
                    
                    # Use enhanced guideline retrieval
                    guidelines_info = await self.guidelines_helper.get_vulnerability_guidelines(
                        building_type or query, 
                        building_age or 20
                    )
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "\n\n".join([node.text for node in retrieval_results])
                
                # # Store in context
                # current_state = await ctx.get("state")
                # if "guidelines_retrieved" not in current_state:
                #     current_state["guidelines_retrieved"] = {}
                
                # current_state["guidelines_retrieved"]["vulnerability"] = guidelines_info
                # await ctx.set("state", current_state)

                # NEW: Log actual guidelines content
                logger.debug(f"Retrieved vulnerability guidelines:\n{guidelines_info}")

                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["vulnerability"] = guidelines_info
                await ctx.set("state", current_state)
                
                logger.info("Successfully retrieved vulnerability guidelines")
                return f"Retrieved vulnerability guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying vulnerability guidelines: {str(e)}")
                return "Error retrieving vulnerability guidelines. Using default assessment only."
        
        # CAT Modeling Agent Tools
        async def evaluate_cat_modeling(ctx: Context, location_data: Dict[str, Any]) -> str:
            """
            Evaluates the catastrophe risk based on location, flood zones, 
            earthquake potential, etc.
            
            Args:
                location_data: Dictionary containing location details
                
            Returns:
                A description of the CAT modeling assessment with score (1-5)
            """
            logger.info("Starting CAT modeling assessment")
            current_state = await ctx.get("state")
            
            try:
                # Use the assessment tools to evaluate CAT risks
                if hasattr(self, 'assessment_tools') and self.assessment_tools:
                    assessment_result = await self.assessment_tools.evaluate_cat_modeling(location_data)
                    cat_score = assessment_result.get("score", 3.0)
                else:
                    # Fallback to original logic if assessment tools aren't available
                    address = location_data.get("address", "")
                    logger.info(f"Analyzing location: {address}")
                    
                    # Mock implementation - would use real data in production
                    # Simulating a flood zone and earthquake risk evaluation
                    if "flood" in address.lower() or "coastal" in address.lower():
                        flood_risk = 4.5
                    else:
                        flood_risk = 2.0
                    
                    # Simple geographic rules - would use proper services in production
                    geo_mapping = {
                        "california": 4.5,  # High earthquake risk
                        "florida": 4.0,     # Hurricane risk
                        "texas": 3.5,       # Multiple hazards
                        "new york": 3.0,
                        "london": 2.0,
                        "birmingham": 1.5,
                        "manchester": 2.0
                    }
                    
                    earthquake_risk = 1.0  # Default low risk
                    for region, risk in geo_mapping.items():
                        if region.lower() in address.lower():
                            earthquake_risk = risk
                            break
                    
                    # Calculate overall CAT score
                    cat_score = max(flood_risk, earthquake_risk)  # Taking worst-case scenario
                    
                    assessment_result = {
                        "score": cat_score,
                        "flood_risk_assessment": f"Flood risk level: {flood_risk:.1f}/5.0",
                        "earthquake_risk_assessment": f"Earthquake/natural disaster risk: {earthquake_risk:.1f}/5.0",
                        "location_analyzed": address
                    }
                
                logger.info(f"Calculated CAT score: {cat_score}")
                
                # Store the result
                if "assessment_results" not in current_state:
                    current_state["assessment_results"] = {}
                
                current_state["assessment_results"]["cat_modeling"] = assessment_result
                
                # Track completion as a list
                if "completed_assessments" not in current_state:
                    current_state["completed_assessments"] = []
                    
                if "cat_modeling" not in current_state["completed_assessments"]:
                    current_state["completed_assessments"].append("cat_modeling")
                
                logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
                await ctx.set("state", current_state)
                
                logger.info("CAT modeling assessment completed and stored in context")
                return f"CAT modeling assessment completed. Overall CAT risk score: {cat_score:.1f}/5.0"
            except Exception as e:
                logger.error(f"Error in CAT modeling: {str(e)}")
                return f"Error in CAT modeling: {str(e)}"
        
        async def query_cat_modeling_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for CAT modeling-related information.
            
            Args:
                query: The search query for CAT modeling-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying CAT modeling guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    location = ""
                    
                    # Try to extract location from query
                    if "in" in query.lower() and "properties" in query.lower():
                        parts = query.lower().split("in")
                        if len(parts) >= 2:
                            location = parts[1].strip()
                    
                    # Use enhanced guideline retrieval
                    guidelines_info = await self.guidelines_helper.get_cat_modeling_guidelines(
                        location or query
                    )
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "\n\n".join([node.text for node in retrieval_results])
                
                # # Store in context
                # current_state = await ctx.get("state")
                # if "guidelines_retrieved" not in current_state:
                #     current_state["guidelines_retrieved"] = {}
                
                # current_state["guidelines_retrieved"]["cat_modeling"] = guidelines_info
                # await ctx.set("state", current_state)

                # NEW: Log actual guidelines content
                logger.debug(f"Retrieved CAT modeling guidelines:\n{guidelines_info}")

                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["cat_modeling"] = guidelines_info
                await ctx.set("state", current_state)
                
                logger.info("Successfully retrieved CAT modeling guidelines")
                return f"Retrieved CAT modeling guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying CAT modeling guidelines: {str(e)}")
                return "Error retrieving CAT modeling guidelines. Using default assessment only."
        
        # async def make_decision(ctx: Context) -> str:
        #     """
        #     Checks if all assessments are complete, then analyzes results and makes a final decision.
        #     If confidence is low, requests human review.
            
        #     Returns:
        #         Decision outcome with explanation
        #     """
        #     logger.info("Starting decision making process")
        #     current_state = await ctx.get("state")
            
        #     # Check if all assessments are complete
        #     completed_assessments = current_state.get("completed_assessments", [])
        #     required_assessments = ["hazard", "vulnerability", "cat_modeling"]
            
        #     missing_assessments = [a for a in required_assessments if a not in completed_assessments]
            
        #     if missing_assessments:
        #         logger.info(f"Cannot make decision yet. Missing assessments: {missing_assessments}")
        #         return f"Cannot make decision yet. The following assessments are still pending: {', '.join(missing_assessments)}"
            
        #     # Check if we already have a final decision (to prevent infinite loops)
        #     if current_state.get("decision", {}).get("final", False):
        #         decision = current_state.get("decision", {}).get("outcome", "UNKNOWN")
        #         reason = current_state.get("decision", {}).get("reason", "No reason provided")
        #         confidence = current_state.get("decision", {}).get("confidence", 0.0)
                
        #         decision_messages = {
        #             "PROCEED_TO_QUOTATION": "Submission approved to proceed to quotation.",
        #             "RECOMMEND_SURVEYOR": "Recommend surveyor assessment before proceeding.",
        #             "REJECT": "Submission rejected."
        #         }
                
        #         logger.info(f"Using existing final decision: {decision}")
        #         return f"Final decision already made: {decision_messages.get(decision, decision)}. Reason: {reason}. Confidence: {confidence:.2f}"
            
        #     try:
        #         # Continue with decision making if all assessments are complete
        #         logger.info("All assessments are complete. Proceeding with decision making.")
        #         assessment_results = current_state.get("assessment_results", {})
        #         submission = current_state.get("submission_data", {})
                
        #         # Use the assessment tools to make the decision if available
        #         if hasattr(self, 'assessment_tools') and self.assessment_tools:
        #             decision_result = await self.assessment_tools.make_underwriting_decision(
        #                 assessment_results, 
        #                 submission
        #             )
        #         else:
        #             # Fallback to original logic if assessment tools aren't available
        #             # Gather scores from assessments
        #             hazard_score = assessment_results.get("hazard", {}).get("score", 3.0)
        #             vulnerability_score = assessment_results.get("vulnerability", {}).get("score", 3.0)
        #             cat_score = assessment_results.get("cat_modeling", {}).get("score", 3.0)
                    
        #             logger.info(f"Assessment scores - Hazard: {hazard_score}, Vulnerability: {vulnerability_score}, CAT: {cat_score}")
                    
        #             # Calculate composite risk score (weighted average)
        #             weights = {
        #                 "hazard": 0.4,
        #                 "vulnerability": 0.3,
        #                 "cat": 0.3
        #             }
                    
        #             composite_score = (
        #                 hazard_score * weights["hazard"] +
        #                 vulnerability_score * weights["vulnerability"] +
        #                 cat_score * weights["cat"]
        #             )
                    
        #             logger.info(f"Calculated composite risk score: {composite_score}")
                    
        #             # Decision logic based on combined risk score
        #             if composite_score <= 2.0:
        #                 decision = "PROCEED_TO_QUOTATION"
        #                 reason = "Risk profile is within acceptable parameters"
        #                 confidence = min(1.0, max(0.0, 1.0 - (composite_score / 5.0)))
        #             elif composite_score <= 3.5:
        #                 decision = "RECOMMEND_SURVEYOR"
        #                 reason = "Risk profile requires additional assessment"
        #                 confidence = 0.8
        #             else:
        #                 decision = "REJECT"
        #                 reason = "Risk profile exceeds acceptable parameters"
        #                 confidence = min(1.0, max(0.0, composite_score / 5.0 - 0.2))
                    
        #             decision_result = {
        #                 "outcome": decision,
        #                 "reason": reason,
        #                 "composite_score": composite_score,
        #                 "confidence": confidence,
        #                 "requires_human_review": confidence < 0.7 or decision == "REJECT",
        #                 "assessment_summary": {
        #                     "hazard_score": hazard_score,
        #                     "vulnerability_score": vulnerability_score,
        #                     "cat_score": cat_score
        #                 }
        #             }
                
        #         logger.info(f"Initial decision: {decision_result['outcome']}, Reason: {decision_result['reason']}, Confidence: {decision_result['confidence']:.2f}")
                
        #         # Human-in-the-loop for decisions with low confidence or rejections
        #         if decision_result.get("requires_human_review", False):
        #             # Check if we already received human feedback (to prevent loops)
        #             if current_state.get("human_feedback_received", False):
        #                 logger.info("Human feedback was already received, using it to finalize decision")
        #                 feedback_comment = current_state.get("human_feedback_comment", "No feedback provided")
        #             else:
        #                 logger.info(f"Decision requires human review. Confidence: {decision_result.get('confidence', 0):.2f}")
                        
        #                 # Important change: Pass decision_result directly to request_human_feedback
        #                 feedback_comment = await request_human_feedback(
        #                     ctx, 
        #                     f"Please review decision: {decision_result['outcome']} for submission {submission.get('submission_id', 'Unknown')}. Confidence: {decision_result.get('confidence', 0):.2f}",
        #                     decision_data=decision_result  # Pass the decision result directly
        #                 )
                        
        #                 # Store the feedback
        #                 current_state["human_feedback_received"] = True
        #                 current_state["human_feedback_comment"] = feedback_comment
                        
        #                 await ctx.set("state", current_state)
        #                 logger.info(f"Human feedback received: {feedback_comment}")
                    
        #             # Modify the decision based on feedback if needed
        #             if "approve" in feedback_comment.lower() or "proceed" in feedback_comment.lower():
        #                 # Override the decision if human approves
        #                 if decision_result["outcome"] == "REJECT":
        #                     decision_result["outcome"] = "PROCEED_TO_QUOTATION" 
        #                     decision_result["reason"] = f"Risk profile approved by human reviewer despite system assessment"
        #                     decision_result["confidence"] = 0.85  # Human override increases confidence
        #                     logger.info("Decision changed to PROCEED_TO_QUOTATION based on human feedback")
        #             elif "surveyor" in feedback_comment.lower() or "survey" in feedback_comment.lower():
        #                 # Change to surveyor recommendation if that's what human suggests
        #                 if decision_result["outcome"] != "RECOMMEND_SURVEYOR":
        #                     decision_result["outcome"] = "RECOMMEND_SURVEYOR"
        #                     decision_result["reason"] = f"Human reviewer recommended additional assessment"
        #                     decision_result["confidence"] = 0.9  # Human override increases confidence
        #                     logger.info("Decision changed to RECOMMEND_SURVEYOR based on human feedback")
        #             elif "reject" in feedback_comment.lower() or "decline" in feedback_comment.lower():
        #                 # Confirm rejection if human agrees
        #                 if decision_result["outcome"] != "REJECT":
        #                     decision_result["outcome"] = "REJECT"
        #                     decision_result["reason"] = f"Human reviewer recommended rejection"
        #                     decision_result["confidence"] = 0.95  # Human override increases confidence
        #                     logger.info("Decision changed to REJECT based on human feedback")
                
        #         # Mark the decision as human reviewed and final
        #         decision_result["human_reviewed"] = current_state.get("human_feedback_received", False)
        #         decision_result["final"] = True
                
        #         # Add eligibility check if using enhanced assessment
        #         if hasattr(self, 'assessment_tools') and self.assessment_tools:
        #             # Check for eligibility issues based on underwriting guidelines
        #             eligibility_issues = []
                    
        #             # Check building age
        #             building_age = datetime.now().year - submission.get("property_details", {}).get("year_built", 0)
        #             if building_age > 35:
        #                 eligibility_issues.append(f"Building age ({building_age} years) exceeds maximum guideline of 35 years")

        #             # Check stories
        #             stories = submission.get("property_details", {}).get("stories", 0)
        #             has_sprinklers = submission.get("property_details", {}).get("sprinklers", False)
        #             max_stories = 3 if has_sprinklers else 2
                    
        #             if stories > max_stories:
        #                 eligibility_issues.append(f"Building stories ({stories}) exceeds maximum guideline of {max_stories} stories")
                        
        #             # Check roof type
        #             construction = submission.get("property_details", {}).get("construction", "").lower()
        #             if "wood shake" in construction:
        #                 eligibility_issues.append(f"Building has ineligible roof type: wood shake")
                    
        #             # Modify decision if there are eligibility issues
        #             if eligibility_issues and decision_result["outcome"] != "REJECT":
        #                 decision_result["outcome"] = "RECOMMEND_SURVEYOR" if decision_result["outcome"] == "PROCEED_TO_QUOTATION" else decision_result["outcome"]
        #                 decision_result["reason"] = f"Eligibility concerns: {'; '.join(eligibility_issues[:2])}"
        #                 decision_result["confidence"] = min(decision_result["confidence"], 0.7)  # Lower confidence due to eligibility issues
        #                 decision_result["eligibility_issues"] = eligibility_issues
                
        #         # Store the final decision
        #         current_state["decision"] = decision_result
        #         await ctx.set("state", current_state)
                
        #         decision_messages = {
        #             "PROCEED_TO_QUOTATION": "Submission approved to proceed to quotation.",
        #             "RECOMMEND_SURVEYOR": "Recommend surveyor assessment before proceeding.",
        #             "REJECT": "Submission rejected."
        #         }
                
        #         logger.info("Decision made and stored in context")
        #         return f"Decision: {decision_messages[decision_result['outcome']]} Reason: {decision_result['reason']}. Confidence: {decision_result['confidence']:.2f}"
        #     except Exception as e:
        #         logger.error(f"Error in decision making: {str(e)}")
        #         return f"Error in decision making: {str(e)}"

        # Add this function to assessment_tools.ipynb under the AssessmentTools class
        async def check_eligibility(submission_data: Dict[str, Any]) -> Tuple[bool, List[str]]:
            """
            Performs explicit eligibility checks based on underwriting guidelines.
            
            Args:
                submission_data: Dictionary containing submission details
                
            Returns:
                Tuple containing (is_eligible, list_of_eligibility_issues)
            """
            eligibility_issues = []
            property_details = submission_data.get("property_details", {})
            
            # Check building age
            year_built = property_details.get("year_built", 0)
            building_age = datetime.now().year - year_built if year_built else 0
            
            if building_age > 35:
                eligibility_issues.append(f"Building age ({building_age} years) exceeds maximum guideline of 35 years")
            
            # Check building type and occupancy
            building_type = property_details.get("building_type", "").lower()
            occupancy = property_details.get("occupancy", "").lower()
            
            # List of ineligible occupancies from the guidelines (Item 35 in General Underwriting Criteria)
            INELIGIBLE_OCCUPANCIES = {
                "amusement", "attorney", "auto filing", "auto parking", "bank", "credit union",
                "bar", "tavern", "bowling", "camp", "carpet stock", "car wash", "church", "synagogue",
                "civic", "fraternal", "club", "hall", "collection", "credit", "loan", "contractor",
                "drive-in", "feed mill", "fraternity", "sorority", "fruit packing", "government",
                "greenhouse", "lodge", "lumber yard", "manufacturing", "nightclub", "newspaper",
                "packing house", "political", "pool hall", "restaurant", "school", "union", "ymca", "ywca",
                "welfare", "woodworker"
            }
            
            # Check for ineligible occupancies
            for ineligible in INELIGIBLE_OCCUPANCIES:
                if ineligible in building_type or ineligible in occupancy:
                    eligibility_issues.append(f"Occupancy type '{occupancy}' or building type '{building_type}' matches ineligible category: '{ineligible}'")
                    break
            
            # Check construction type
            construction = property_details.get("construction", "").lower()
            if "wood shake" in construction or "shake roof" in construction:
                eligibility_issues.append("Buildings with wood shake roofs are ineligible for coverage")
            
            # Check number of stories
            stories = property_details.get("stories", 0)
            has_sprinklers = property_details.get("sprinklers", False)
            
            if stories > 3:
                eligibility_issues.append(f"Buildings exceeding 3 stories are ineligible for coverage")
            elif stories > 2 and not has_sprinklers:
                eligibility_issues.append(f"Buildings over 2 stories must be fully sprinklered")
            
            # Check vacancy
            vacancy = property_details.get("vacancy_percentage", 0)
            if vacancy > 25:
                eligibility_issues.append(f"Buildings with more than 25% vacancy are ineligible for coverage")
            
            # Check protection class
            protection_class = property_details.get("protection_class", 0)
            if protection_class > 6:
                eligibility_issues.append(f"Property must be within Protection Classes 1-6")
            
            # Return eligibility status
            is_eligible = len(eligibility_issues) == 0
            
            return is_eligible, eligibility_issues

        # Replace the make_decision function in the InsuranceAgentSystem.create_agents method
        async def make_decision(ctx: Context) -> str:
            """
            Enhanced decision-making function that incorporates explicit eligibility checks
            and improved guideline retrieval.
            
            Returns:
                Decision outcome with explanation
            """
            logger.info("Starting decision making process")
            current_state = await ctx.get("state")
            
            # Check if all assessments are complete
            completed_assessments = current_state.get("completed_assessments", [])
            required_assessments = ["hazard", "vulnerability", "cat_modeling"]
            
            missing_assessments = [a for a in required_assessments if a not in completed_assessments]
            
            if missing_assessments:
                logger.info(f"Cannot make decision yet. Missing assessments: {missing_assessments}")
                return f"Cannot make decision yet. The following assessments are still pending: {', '.join(missing_assessments)}"
            
            # Check if we already have a final decision (to prevent infinite loops)
            if current_state.get("decision", {}).get("final", False):
                decision = current_state.get("decision", {}).get("outcome", "UNKNOWN")
                reason = current_state.get("decision", {}).get("reason", "No reason provided")
                confidence = current_state.get("decision", {}).get("confidence", 0.0)
                
                decision_messages = {
                    "PROCEED_TO_QUOTATION": "Submission approved to proceed to quotation.",
                    "RECOMMEND_SURVEYOR": "Recommend surveyor assessment before proceeding.",
                    "REJECT": "Submission rejected."
                }
                
                logger.info(f"Using existing final decision: {decision}")
                return f"Final decision already made: {decision_messages.get(decision, decision)}. Reason: {reason}. Confidence: {confidence:.2f}"
            
            try:
                # Continue with decision making if all assessments are complete
                logger.info("All assessments are complete. Proceeding with decision making.")
                assessment_results = current_state.get("assessment_results", {})
                submission_data = current_state.get("submission_data", {})
                
                # Get relevant decision guidelines
                decision_guidelines = await query_decision_guidelines(ctx, submission_data)
                
                # Check eligibility based on explicit criteria
                is_eligible, eligibility_issues = await check_eligibility(submission_data)
                
                # If not eligible, decision is made
                if not is_eligible:
                    # If there are serious eligibility issues, reject outright
                    serious_issues = [issue for issue in eligibility_issues if 
                                    "ineligible" in issue.lower() or
                                    "exceeds" in issue.lower()]
                    
                    if serious_issues:
                        decision = "REJECT"
                        reason = f"Eligibility issues: {'; '.join(serious_issues[:2])}"
                        confidence = 0.95
                    else:
                        # For less serious issues, recommend surveyor
                        decision = "RECOMMEND_SURVEYOR" 
                        reason = f"Eligibility concerns: {'; '.join(eligibility_issues[:2])}"
                        confidence = 0.85
                    
                    decision_result = {
                        "outcome": decision,
                        "reason": reason,
                        "composite_score": 4.0,  # High risk score for ineligible submissions
                        "confidence": confidence,
                        "requires_human_review": confidence < 0.9,
                        "eligibility_issues": eligibility_issues,
                        "assessment_summary": {
                            "is_eligible": is_eligible,
                            "eligibility_issues": eligibility_issues
                        }
                    }
                else:
                    # Submission is eligible, proceed with normal decision logic
                    # Gather scores from assessments
                    hazard_score = assessment_results.get("hazard", {}).get("score", 3.0)
                    vulnerability_score = assessment_results.get("vulnerability", {}).get("score", 3.0)
                    cat_score = assessment_results.get("cat_modeling", {}).get("score", 3.0)
                    
                    logger.info(f"Assessment scores - Hazard: {hazard_score}, Vulnerability: {vulnerability_score}, CAT: {cat_score}")
                    
                    # Calculate composite risk score (weighted average)
                    weights = {
                        "hazard": 0.4,
                        "vulnerability": 0.3,
                        "cat": 0.3
                    }
                    
                    composite_score = (
                        hazard_score * weights["hazard"] +
                        vulnerability_score * weights["vulnerability"] +
                        cat_score * weights["cat"]
                    )
                    
                    logger.info(f"Calculated composite risk score: {composite_score}")
                    
                    # Decision logic based on combined risk score - with adjusted thresholds
                    if composite_score <= 1.5:
                        decision = "PROCEED_TO_QUOTATION"
                        reason = "Risk profile is excellent and well within acceptable parameters"
                        confidence = 0.90  # High confidence for very good submissions
                    elif composite_score <= 2.0:
                        decision = "PROCEED_TO_QUOTATION"
                        reason = "Risk profile is within acceptable parameters"
                        confidence = min(1.0, max(0.0, 0.8 - (composite_score - 1.5) * 0.4))  # Scales from 0.8 to 0.6
                    elif composite_score <= 3.0:
                        decision = "RECOMMEND_SURVEYOR"
                        reason = "Risk profile requires additional assessment"
                        confidence = 0.8
                    else:
                        decision = "REJECT"
                        reason = "Risk profile exceeds acceptable parameters"
                        confidence = min(1.0, max(0.0, 0.7 + (composite_score - 3.0) * 0.1))  # Higher confidence for clear rejections
                    
                    decision_result = {
                        "outcome": decision,
                        "reason": reason,
                        "composite_score": composite_score,
                        "confidence": confidence,
                        # "requires_human_review": confidence < 0.75 or decision == "REJECT",
                        "requires_human_review": decision in ["REJECT", "RECOMMEND_SURVEYOR"] or confidence < 0.65,
                        "assessment_summary": {
                            "hazard_score": hazard_score,
                            "vulnerability_score": vulnerability_score,
                            "cat_score": cat_score,
                            "is_eligible": is_eligible,
                            "eligibility_issues": eligibility_issues
                        }
                    }
                
                # Human-in-the-loop for decisions with low confidence or rejections
                if decision_result.get("requires_human_review", False):
                    # Check if we already received human feedback (to prevent loops)
                    if current_state.get("human_feedback_received", False):
                        logger.info("Human feedback was already received, using it to finalize decision")
                        feedback_comment = current_state.get("human_feedback_comment", "No feedback provided")
                    else:
                        logger.info(f"Decision requires human review. Confidence: {decision_result.get('confidence', 0):.2f}")
                        
                        # Pass decision_result directly to request_human_feedback
                        feedback_comment = await request_human_feedback(
                            ctx, 
                            f"Please review decision: {decision_result['outcome']} for submission {submission_data.get('submission_id', 'Unknown')}. Confidence: {decision_result.get('confidence', 0):.2f}",
                            decision_data=decision_result  # Pass the decision result directly
                        )
                        
                        # Store the feedback
                        current_state["human_feedback_received"] = True
                        current_state["human_feedback_comment"] = feedback_comment
                        
                        await ctx.set("state", current_state)
                        logger.info(f"Human feedback received: {feedback_comment}")
                    
                    # Modify the decision based on feedback if needed
                    if "approve" in feedback_comment.lower() or "proceed" in feedback_comment.lower():
                        # Override the decision if human approves
                        if decision_result["outcome"] == "REJECT":
                            decision_result["outcome"] = "PROCEED_TO_QUOTATION" 
                            
                            decision_result["reason"] = f"Risk profile approved by human reviewer despite system assessment"
                            decision_result["confidence"] = 0.85  # Human override increases confidence
                            logger.info("Decision changed to PROCEED_TO_QUOTATION based on human feedback")
                    elif "surveyor" in feedback_comment.lower() or "survey" in feedback_comment.lower():
                        # Change to surveyor recommendation if that's what human suggests
                        if decision_result["outcome"] != "RECOMMEND_SURVEYOR":
                            decision_result["outcome"] = "RECOMMEND_SURVEYOR"
                            decision_result["reason"] = f"Human reviewer recommended additional assessment"
                            decision_result["confidence"] = 0.9  # Human override increases confidence
                            logger.info("Decision changed to RECOMMEND_SURVEYOR based on human feedback")
                    elif "reject" in feedback_comment.lower() or "decline" in feedback_comment.lower():
                        # Confirm rejection if human agrees
                        if decision_result["outcome"] != "REJECT":
                            decision_result["outcome"] = "REJECT"
                            decision_result["reason"] = f"Human reviewer recommended rejection"
                            decision_result["confidence"] = 0.95  # Human override increases confidence
                            logger.info("Decision changed to REJECT based on human feedback")
                
                # Mark the decision as human reviewed and final
                decision_result["human_reviewed"] = current_state.get("human_feedback_received", False)
                decision_result["final"] = True
                
                # Store the final decision
                current_state["decision"] = decision_result
                await ctx.set("state", current_state)
                
                decision_messages = {
                    "PROCEED_TO_QUOTATION": "Submission approved to proceed to quotation.",
                    "RECOMMEND_SURVEYOR": "Recommend surveyor assessment before proceeding.",
                    "REJECT": "Submission rejected."
                }
                
                logger.info("Decision made and stored in context")
                return f"Decision: {decision_messages[decision_result['outcome']]} Reason: {decision_result['reason']}. Confidence: {decision_result['confidence']:.2f}"
            except Exception as e:
                logger.error(f"Error in decision making: {str(e)}")
                return f"Error in decision making: {str(e)}"

        async def query_decision_guidelines(ctx: Context, query: str) -> str:
            """
            Queries the underwriting guidelines for decision-making information.
            
            Args:
                query: The search query for decision-related guidelines
                
            Returns:
                Relevant guidelines information
            """
            logger.info(f"Querying decision guidelines with: {query}")
            
            if not hasattr(self, 'guidelines_retriever') or self.guidelines_retriever is None:
                return "Guidelines retriever not available. Using default guidelines."
            
            try:
                if hasattr(self, 'guidelines_helper') and self.guidelines_helper:
                    # Use enhanced guideline retrieval
                    submission_data = (await ctx.get("state")).get("submission_data", {})
                    guidelines_info = await self.guidelines_helper.get_decision_guidelines(submission_data)
                else:
                    # Fallback to basic retrieval
                    retrieval_results = self.guidelines_retriever.retrieve(query)
                    guidelines_info = "\n\n".join([node.text for node in retrieval_results])
                
                # # Store in context
                # current_state = await ctx.get("state")
                # if "guidelines_retrieved" not in current_state:
                #     current_state["guidelines_retrieved"] = {}
                
                # current_state["guidelines_retrieved"]["decision"] = guidelines_info
                # await ctx.set("state", current_state)

                # NEW: Log actual guidelines content
                logger.debug(f"Retrieved decision guidelines:\n{guidelines_info}")

                # Store in context
                current_state = await ctx.get("state")
                current_state.setdefault("guidelines_retrieved", {})["decision"] = guidelines_info
                await ctx.set("state", current_state)
                
                logger.info("Successfully retrieved decision guidelines")
                return f"Retrieved decision guidelines: {guidelines_info}"
            except Exception as e:
                logger.error(f"Error querying decision guidelines: {str(e)}")
                return "Error retrieving decision guidelines. Using default assessment logic only."
        
        # Communication Agent Tools
        async def send_notification(ctx: Context) -> str:
            """
            Formats and sends notification email based on the decision.
            
            Returns:
                Confirmation of notification sent
            """
            logger.info("Starting notification preparation")
            current_state = await ctx.get("state")
            decision = current_state.get("decision", {})

            # Check if this is a final decision
            if not decision.get("final", False):
                return "Cannot send notification for a non-final decision. Decision requires human review first."

            # Get the decision outcome and reason
            submission = current_state.get("submission_data", {})
            
            try:
                # Format the notification using assessment tools if available
                if hasattr(self, 'assessment_tools') and self.assessment_tools:
                    notification = self.assessment_tools.format_notification(decision, submission)
                else:
                    # Get the decision outcome and reason
                    outcome = decision.get("outcome", "UNKNOWN")
                    reason = decision.get("reason", "No reason provided")
                    
                    logger.info(f"Preparing notification for decision: {outcome}")
                    
                    # Basic email templates
                    email_templates = {
                        "PROCEED_TO_QUOTATION": """
Subject: Submission {submission_id} Approved for Quotation

Dear Distribution Team,

The submission for {insured_name} (ID: {submission_id}) has been reviewed and approved to proceed to quotation.

Risk Assessment Summary:
- Hazard Score: {hazard_score}/5.0
- Vulnerability Score: {vulnerability_score}/5.0
- CAT Risk Score: {cat_score}/5.0

Decision: Proceed to Quotation
Confidence: {confidence:.0%}

Please proceed with the quotation process.

Regards,
Underwriting AI Assistant
""",
                        "RECOMMEND_SURVEYOR": """
Subject: Submission {submission_id} Requires Surveyor Assessment

Dear Distribution Team,

The submission for {insured_name} (ID: {submission_id}) has been reviewed and requires a surveyor assessment before proceeding.

Risk Assessment Summary:
- Hazard Score: {hazard_score}/5.0
- Vulnerability Score: {vulnerability_score}/5.0
- CAT Risk Score: {cat_score}/5.0

Reason for surveyor recommendation: {reason}
Confidence: {confidence:.0%}

Please arrange for a risk assessment survey.

Regards,
Underwriting AI Assistant
""",
                        "REJECT": """
Subject: Submission {submission_id} Rejected

Dear Distribution Team,

The submission for {insured_name} (ID: {submission_id}) has been reviewed and cannot proceed.

Risk Assessment Summary:
- Hazard Score: {hazard_score}/5.0
- Vulnerability Score: {vulnerability_score}/5.0
- CAT Risk Score: {cat_score}/5.0

Reason for rejection: {reason}
Confidence: {confidence:.0%}

If you have additional information that might change this assessment, please provide it.

Regards,
Underwriting AI Assistant
"""
                    }
                    
                    # Get template for current decision
                    template = email_templates.get(outcome, "Unknown decision type")
                    
                    # Fill in template variables
                    assessment_summary = decision.get("assessment_summary", {})
                    email_content = template.format(
                        submission_id=submission.get("submission_id", "Unknown"),
                        insured_name=submission.get("insured_name", "Unknown"),
                        hazard_score=assessment_summary.get("hazard_score", 0),
                        vulnerability_score=assessment_summary.get("vulnerability_score", 0),
                        cat_score=assessment_summary.get("cat_score", 0),
                        reason=reason,
                        confidence=decision.get("confidence", 0.0)
                    )
                    
                    notification = {
                        "recipient": "distribution_team@company.com",
                        "content": email_content,
                        "sent": True,
                        "timestamp": datetime.now().isoformat()
                    }
                
                # Log the actual email content
                logger.info(f"Email Body:\n{notification['content']}")
                # Include eligibility issues if present
                if decision.get("eligibility_issues") and "content" in notification:
                    eligibility_issues = decision.get("eligibility_issues", [])
                    eligibility_section = "\nEligibility Issues:\n" + "\n".join([f"- {issue}" for issue in eligibility_issues])
                    notification["content"] = notification["content"].replace("Regards,", eligibility_section + "\n\nRegards,")
                
                # Store the notification in the state
                current_state["notification"] = notification
                
                # Mark workflow as complete
                current_state["workflow_complete"] = True
                await ctx.set("state", current_state)
                
                # In production, this would actually send the email
                logger.info(f"Email notification prepared for distribution team regarding submission {submission.get('submission_id', 'Unknown')}")
                
                return f"Notification email prepared and ready to send for decision: {decision.get('outcome', 'Unknown')}"
            except Exception as e:
                logger.error(f"Error preparing notification: {str(e)}")
                return f"Error preparing notification: {str(e)}"
        
        # This patch fixes the issue in the request_human_feedback function
        # The problem is that the decision data isn't fully stored in the context yet
        # when the function is called, resulting in placeholder values being displayed

        # async def request_human_feedback(ctx: Context, question: str = "", decision_data: Dict[str, Any] = None) -> str:
        #     """
        #     Requests human feedback using LlamaIndex's event system.
            
        #     Args:
        #         ctx: Context object
        #         question: Question to ask the human reviewer
        #         decision_data: Optional decision data directly from the decision calculation
                
        #     Returns:
        #         Human feedback response
        #     """
        #     logger.info(f"Requesting human feedback: {question}")
        #     current_state = await ctx.get("state")
            
        #     # Get decision and submission details for context
        #     # Use directly provided decision_data if available, otherwise fall back to context
        #     decision = decision_data if decision_data else current_state.get("decision", {})
        #     submission = current_state.get("submission_data", {})
        #     assessment_results = current_state.get("assessment_results", {})
            
        #     # Get relevant guidelines information
        #     guidelines_info = current_state.get("guidelines_retrieved", {}).get("decision", "No specific guidelines retrieved")
            
        #     # Extract decision details, ensuring we have values even if not finalized
        #     outcome = decision.get("outcome", "Unknown")
        #     reason = decision.get("reason", "No reason provided")
        #     composite_score = decision.get("composite_score", 0)
        #     confidence = decision.get("confidence", 0)
            
        #     # Add eligibility issues if present
        #     eligibility_issues_text = ""
        #     if decision.get("eligibility_issues"):
        #         eligibility_issues_text = "\nEligibility Issues:\n" + "\n".join([f"- {issue}" for issue in decision.get("eligibility_issues", [])])
            
        #     # Create a detailed prompt for the human reviewer
        #     detailed_question = f"""
        # HUMAN REVIEW REQUIRED:
        # {question}

        # Submission: {submission.get('submission_id')} - {submission.get('insured_name')}
        # Property: {submission.get('property_details', {}).get('building_type')} at {submission.get('property_details', {}).get('address')}

        # Assessment Results:
        # - Hazard: {assessment_results.get('hazard', {}).get('score', 0)}/5.0
        # - Vulnerability: {assessment_results.get('vulnerability', {}).get('score', 0)}/5.0
        # - CAT Risk: {assessment_results.get('cat_modeling', {}).get('score', 0)}/5.0
        # - Composite Score: {composite_score:.1f}/5.0{eligibility_issues_text}

        # System Decision: {outcome}
        # Reason: {reason}
        # Confidence: {confidence:.0%}

        # Relevant Guidelines:
        # {guidelines_info}

        # Please review and provide feedback. Type 'approve' to confirm the decision, or provide specific guidance:
        # """
            
        #     # Use LlamaIndex's wait_for_event to get human input
        #     response = await ctx.wait_for_event(
        #         HumanResponseEvent,
        #         waiter_id=question,
        #         waiter_event=InputRequiredEvent(
        #             prefix=detailed_question,
        #             user_name="Underwriter",
        #         ),
        #         requirements={"user_name": "Underwriter"},
        #     )
            
        #     # Record the feedback in the state
        #     if "human_feedback" not in current_state:
        #         current_state["human_feedback"] = {}
            
        #     current_state["human_feedback"]["requested"] = True
        #     current_state["human_feedback"]["timestamp"] = datetime.now().isoformat()
        #     current_state["human_feedback"]["comment"] = response.response
        #     current_state["human_feedback"]["status"] = "completed"
            
        #     await ctx.set("state", current_state)
            
        #     return response.response

        # Update the request_human_feedback function in InsuranceAgentSystem.create_agents method
        # 
        async def request_human_feedback(ctx: Context, question: str = "", decision_data: Dict[str, Any] = None) -> str:
            """
            Requests human feedback using LlamaIndex's event system.

            Args:
                ctx: Context object
                question: Question to ask the human reviewer
                decision_data: Optional decision data directly from the decision calculation

            Returns:
                Human feedback response
            """
            logger.info(f"Requesting human feedback: {question}")
            current_state = await ctx.get("state")

            # Use directly provided decision_data if available, otherwise fall back to context
            decision = decision_data if decision_data else current_state.get("decision", {})
            submission = current_state.get("submission_data", {})
            assessment_results = current_state.get("assessment_results", {})

            # Extract and trim guideline info for readability
            raw_guidelines = current_state.get("guidelines_retrieved", {}).get("decision", "")
            lines = raw_guidelines.strip().splitlines()
            clean_lines = [line.strip() for line in lines if line.strip()]
            top_guidelines = "\n".join(clean_lines[:10])  # Show only top 10 lines
            guidelines_info = top_guidelines or "No specific guidelines retrieved"

            # Extract decision details
            outcome = decision.get("outcome", "Unknown")
            reason = decision.get("reason", "No reason provided")
            composite_score = decision.get("composite_score", 0)
            confidence = decision.get("confidence", 0)

            # Add eligibility issues if present
            eligibility_issues_text = ""
            if decision.get("eligibility_issues"):
                eligibility_issues_text = "\nEligibility Issues:\n" + "\n".join(
                    [f"- {issue}" for issue in decision.get("eligibility_issues", [])]
                )

            # Compose the human-readable question
            detailed_question = f"""
        🚨 HUMAN REVIEW REQUIRED 🚨

        🔍 Question:
        {question}

        📄 Submission:
        - ID: {submission.get('submission_id')}
        - Insured: {submission.get('insured_name')}
        - Property: {submission.get('property_details', {}).get('building_type')} at {submission.get('property_details', {}).get('address')}

        📊 Assessment Summary:
        - Hazard: {round(assessment_results.get('hazard', {}).get('score', 0), 1)}/5.0
        - Vulnerability: {round(assessment_results.get('vulnerability', {}).get('score', 0), 1)}/5.0
        - CAT Risk: {round(assessment_results.get('cat_modeling', {}).get('score', 0), 1)}/5.0
        - Composite Score: {composite_score}/5.0{eligibility_issues_text}

        🧠 System Decision: {outcome}
        📌 Reason: {reason}
        🎯 Confidence: {confidence:.0%}

        📚 Key Guideline Excerpts:
        {guidelines_info}

        💬 Please type 'approve', 'reject', or provide specific guidance:
        """

            # Request human feedback using LlamaIndex's event system
            response = await ctx.wait_for_event(
                HumanResponseEvent,
                waiter_id=question,
                waiter_event=InputRequiredEvent(
                    prefix=detailed_question,
                    user_name="Underwriter",
                ),
                requirements={"user_name": "Underwriter"},
            )

            # Record the feedback in context
            if "human_feedback" not in current_state:
                current_state["human_feedback"] = {}

            current_state["human_feedback"]["requested"] = True
            current_state["human_feedback"]["timestamp"] = datetime.now().isoformat()
            current_state["human_feedback"]["comment"] = response.response
            current_state["human_feedback"]["status"] = "completed"

            await ctx.set("state", current_state)

            return response.response

        
        # Create the agents
        self.hazard_agent = FunctionAgent(
            name="HazardClassificationAgent",
            description="Evaluates the hazard classification of properties based on building type, construction, and occupancy.",
            system_prompt=(
                "You are a specialist in evaluating property hazard classifications for insurance purposes. "
                "You analyze building types, construction materials, and occupancy types to determine the risk level. "
                "First, query the underwriting guidelines for hazard-related criteria. "
                "Then, evaluate the submission using both the guidelines and your standard assessment criteria. "
                "Be thorough and precise in your assessments, using standard insurance industry criteria. "
                "After completing your assessment, hand off to the VulnerabilityAssessmentAgent."
            ),
            llm=self.llm,
            tools=[evaluate_hazard_classification, query_hazard_guidelines],
            can_handoff_to=["VulnerabilityAssessmentAgent"]
        )

        self.vulnerability_agent = FunctionAgent(
            name="VulnerabilityAssessmentAgent",
            description="Evaluates property vulnerability based on security systems and protective measures.",
            system_prompt=(
                "You are a specialist in evaluating property vulnerabilities for insurance purposes. "
                "You assess security systems, fire protection measures, and other safety features to determine the risk level. "
                "First, query the underwriting guidelines for vulnerability-related criteria. "
                "Then, evaluate the submission using both the guidelines and your standard vulnerability assessment. "
                "Be thorough in considering all protective measures and their effectiveness. "
                "After completing your assessment, hand off to the CATModelingAgent."
            ),
            llm=self.llm,
            tools=[evaluate_vulnerability, query_vulnerability_guidelines],
            can_handoff_to=["CATModelingAgent"]
        )

        self.cat_modeling_agent = FunctionAgent(
            name="CATModelingAgent",
            description="Evaluates catastrophe risks based on geographical location and environmental factors.",
            system_prompt=(
                "You are a specialist in catastrophe risk modeling for insurance purposes. "
                "You analyze geographical locations to assess risks from natural disasters like floods, earthquakes, and storms. "
                "First, query the underwriting guidelines for catastrophe risk-related criteria. "
                "Then, evaluate the location using both the guidelines and standard geographical risk assessment. "
                "Use precise geographical data and historical patterns to determine risk levels. "
                "After completing your assessment, hand off to the DecisionAgent."
            ),
            llm=self.llm,
            tools=[evaluate_cat_modeling, query_cat_modeling_guidelines],
            can_handoff_to=["DecisionAgent"]
        )

        self.decision_agent = FunctionAgent(
            name="DecisionAgent",
            description="Analyzes all assessment results and makes the final underwriting decision.",
            system_prompt=(
                "You are a decision specialist for insurance underwriting. "
                "You analyze assessment results from multiple domains to determine whether to proceed with quotation, "
                "recommend a surveyor, or reject a submission. "
                "First, check that all required assessments (hazard, vulnerability, cat modeling) have been completed. "
                "Then, query the underwriting guidelines for decision criteria. "
                "Combine the assessment results with the guidelines to make a comprehensive decision. "
                "Balance all risk factors to make the most appropriate decision. "
                "For low confidence decisions or rejections, request human review. "
                "After making your decision, hand off to the CommunicationAgent."
            ),
            llm=self.llm,
            tools=[make_decision, query_decision_guidelines],
            can_handoff_to=["CommunicationAgent"]
        )

        self.communication_agent = FunctionAgent(
            name="CommunicationAgent",
            description="Formats and sends notifications based on the underwriting decision.",
            system_prompt=(
                "You are a communication specialist for insurance operations. "
                "You create clear, appropriate notifications to inform stakeholders about underwriting decisions. "
                "Ensure all relevant information is included in the appropriate format. "
                "After sending the notification, the decision process is complete."
            ),
            llm=self.llm,
            tools=[send_notification]
        )
    
    def create_workflow(self, submission_data: Dict[str, Any]) -> AgentWorkflow:
        """
        Create the workflow with all agents
        
        Args:
            submission_data: The insurance submission data to evaluate
            
        Returns:
            AgentWorkflow object ready to run
        """
        # Create the workflow with the 4 essential agents
        workflow = AgentWorkflow(
            agents=[
                self.hazard_agent,
                self.vulnerability_agent,
                self.cat_modeling_agent,
                self.decision_agent,
                self.communication_agent
            ],
            # Start with hazard agent
            root_agent=self.hazard_agent.name,
            initial_state={
                "submission_data": submission_data,
                "assessment_results": {},
                "completed_assessments": [],
                "guidelines_retrieved": {},
                "decision": {},
                "notification": {},
                "workflow_complete": False
            }
        )
        
        logger.info("Agent workflow created successfully")
        return workflow
    
    async def run_workflow(self, submission_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Run the multi-agent workflow with human-in-the-loop capability
        
        Args:
            submission_data: The insurance submission data to evaluate
            
        Returns:
            The final state after workflow completion
        """
        workflow = self.create_workflow(submission_data)
        
        logger.info("Starting workflow execution...")
        
        # Run the workflow with a more explicit message
        handler = workflow.run(
            user_msg=(
                "Please process this insurance submission through the full assessment workflow. "
                "For each step, first query the relevant underwriting guidelines, then perform your assessment. "
                "Begin with the hazard classification, then evaluate vulnerability, "
                "perform CAT modeling, make a decision, and finally send the appropriate notification."
            )
        )
        
        # Stream the output to see progress
        current_agent = None
        
        async for event in handler.stream_events():
            # Check if workflow is complete after each step
            current_state = await handler.ctx.get("state")
            if current_state.get("workflow_complete", False):
                logger.info("Workflow has been marked as complete, stopping further processing.")
                break
                
            if hasattr(event, "current_agent_name") and event.current_agent_name != current_agent:
                current_agent = event.current_agent_name
                logger.info(f"==== Agent: {current_agent} ====")
                
            # Handle human input requests
            if isinstance(event, InputRequiredEvent):
                print("\n" + "="*50)
                print("\nHUMAN REVIEW REQUIRED")
                print("="*50)
                user_input = input(event.prefix + "\n> ")
                print("="*50 + "\n")
                
                # Send the human response back to the workflow
                handler.ctx.send_event(
                    HumanResponseEvent(
                        response=user_input,
                        user_name=event.user_name,
                    )
                )
            
            # Check if a final decision has been made
            if current_state.get("decision", {}).get("final", False) and current_agent == "CommunicationAgent":
                logger.info("Final decision has been made and notification sent. Workflow will complete.")
                current_state["workflow_complete"] = True
                await handler.ctx.set("state", current_state)
        
        # Get the final state
        final_state = await handler.ctx.get("state")
        
        # Log the results
        logger.info("Workflow execution completed.")
        logger.info(f"Decision: {final_state.get('decision', {}).get('outcome', 'No decision')}")
        logger.info(f"Reason: {final_state.get('decision', {}).get('reason', 'No reason provided')}")
        
        # Return the final state
        return final_state
    
    def summarize_results(self, final_state: Dict[str, Any]) -> str:
        """
        Generate a human-readable summary of the workflow results
        
        Args:
            final_state: The final state after workflow completion
            
        Returns:
            Formatted string with summary information
        """
        decision = final_state.get("decision", {})
        submission = final_state.get("submission_data", {})
        
        summary = []
        summary.append("\n====== INSURANCE SUBMISSION ASSESSMENT RESULTS ======")
        summary.append(f"Submission ID: {submission.get('submission_id', 'Unknown')}")
        summary.append(f"Insured: {submission.get('insured_name', 'Unknown')}")
        
        # Add property details
        property_details = submission.get("property_details", {})
        if property_details:
            summary.append(f"\nPROPERTY DETAILS:")
            summary.append(f"- Address: {property_details.get('address', 'Unknown')}")
            summary.append(f"- Building Type: {property_details.get('building_type', 'Unknown')}")
            summary.append(f"- Construction: {property_details.get('construction', 'Unknown')}")
            summary.append(f"- Year Built: {property_details.get('year_built', 'Unknown')}")
            summary.append(f"- Sprinklers: {'Yes' if property_details.get('sprinklers', False) else 'No'}")
        
        summary.append("\nCOMPLETED ASSESSMENTS:")
        completed = ", ".join(final_state.get("completed_assessments", []))
        summary.append(f"- {completed}")
        
        summary.append("\nRISK ASSESSMENT SUMMARY:")
        assessment_summary = decision.get("assessment_summary", {})
        summary.append(f"- Hazard Score: {assessment_summary.get('hazard_score', 0):.1f}/5.0")
        summary.append(f"- Vulnerability Score: {assessment_summary.get('vulnerability_score', 0):.1f}/5.0")
        summary.append(f"- CAT Risk Score: {assessment_summary.get('cat_score', 0):.1f}/5.0")
        summary.append(f"- Composite Score: {decision.get('composite_score', 0):.1f}/5.0")
        
        # Add eligibility issues if present
        eligibility_issues = decision.get("eligibility_issues", [])
        if eligibility_issues:
            summary.append("\nELIGIBILITY CONCERNS:")
            for issue in eligibility_issues:
                summary.append(f"- {issue}")
        
        summary.append(f"\nDECISION: {decision.get('outcome', 'Unknown')}")
        summary.append(f"Reason: {decision.get('reason', 'No reason provided')}")
        summary.append(f"Confidence: {decision.get('confidence', 0):.0%}")
        
        # Show human feedback if available
        human_feedback = final_state.get("human_feedback", {})
        if human_feedback:
            summary.append("\nHUMAN FEEDBACK:")
            summary.append(f"Status: {human_feedback.get('status', 'Not provided')}")
            summary.append(f"Comment: {human_feedback.get('comment', 'No comment')}")
        
        # Add notification details if available
        notification = final_state.get("notification", {})
        if notification:
            summary.append("\nNOTIFICATION STATUS:")
            summary.append(f"Recipient: {notification.get('recipient', 'Unknown')}")
            summary.append(f"Sent: {'Yes' if notification.get('sent', False) else 'No'}")
            summary.append(f"Timestamp: {notification.get('timestamp', 'Unknown')}")
        
        # Add reference to guidelines used
        guidelines_referenced = decision.get("guidelines_referenced", "")
        if guidelines_referenced and len(guidelines_referenced) > 0:
            summary.append("\nREFERENCED GUIDELINES:")
            summary.append(f"{guidelines_referenced[:300]}...")
        
        summary.append("\n===========================================")
        
        return "\n".join(summary)

In [14]:
# Add these test cases to your project
# Test Case 1: Clear Approval Case
clear_approval_submission = {
    "submission_id": "SUB20250501001",
    "broker_name": "Premier Insurance Partners",
    "insured_name": "TechOffice Solutions Inc.",
    "property_details": {
        "address": "45 Corporate Plaza, Palo Alto, CA 94304",
        "building_type": "Office",
        "construction": "Concrete and Steel",
        "year_built": 2010,  # 15 years old
        "area_sqm": 3500,
        "stories": 2,
        "occupancy": "Professional Office Space",
        "sprinklers": True,
        "alarm_system": "Grade A - 24hr Monitored with Video",
        "protection_class": 2,  # Excellent fire protection
        "vacancy_percentage": 5  # Low vacancy rate
    },
    "coverage": {
        "building_value": 5200000.0,
        "contents_value": 1800000.0,
        "business_interruption": 2500000.0,
        "deductible": 25000.0
    },
    "client_history": {
        "claims_count": 0,  # No claims
        "previous_policies": 4,  # Good history
        "risk_score": 82  # High score (good)
    }
}

# Test Case 2: Borderline Case (Requiring Human Review)
borderline_submission = {
    "submission_id": "SUB20250501002",
    "broker_name": "ABC Insurance Brokers",
    "insured_name": "Precision Parts Manufacturing Ltd.",
    "property_details": {
        "address": "789 Industrial Parkway, Fresno, CA 93706",
        "building_type": "Manufacturing",
        "construction": "Steel Frame with Brick",
        "year_built": 1995,  # 30 years old
        "area_sqm": 7500,
        "stories": 2,
        "occupancy": "Light Manufacturing - Plastic Components",
        "sprinklers": True,
        "alarm_system": "Basic Monitored System",  # Basic system only
        "protection_class": 4,  # Average fire protection
        "vacancy_percentage": 15  # Moderate vacancy
    },
    "coverage": {
        "building_value": 6200000.0,
        "contents_value": 3500000.0,
        "business_interruption": 4000000.0,
        "deductible": 50000.0
    },
    "client_history": {
        "claims_count": 1,  # One previous claim
        "previous_policies": 3,
        "risk_score": 65  # Average score
    }
}

# Test Case 3: Likely Rejection Case
likely_rejection_submission = {
    "submission_id": "SUB20250501003",
    "broker_name": "Regional Insurance Group",
    "insured_name": "Budget Storage Solutions Inc.",
    "property_details": {
        "address": "1250 Warehouse Blvd, Sacramento, CA 95834",
        "building_type": "Warehouse",
        "construction": "Partially Wood Frame with Metal Siding",  # Wood construction elements
        "year_built": 1982,  # 43 years old - exceeds 35 year limit
        "area_sqm": 12000,
        "stories": 1,
        "occupancy": "Storage Warehouse with Vehicle Parts",
        "sprinklers": False,  # No sprinklers
        "alarm_system": "Local Alarm Only",  # Basic alarm only
        "protection_class": 6,  # Borderline acceptable
        "vacancy_percentage": 30,  # Exceeds 25% vacancy limit
        "maintenance_issues": "Electrical systems need updating, roof leaks in multiple areas"
    },
    "coverage": {
        "building_value": 4800000.0,
        "contents_value": 7500000.0,
        "business_interruption": 2000000.0,
        "deductible": 100000.0
    },
    "client_history": {
        "claims_count": 3,  # Multiple claims
        "previous_policies": 1,  # Limited history
        "risk_score": 48  # Low score (bad)
    },
    "additional_risk_factors": {
        "flood_zone": "High Risk",
        "electrical_inspection_status": "Failed - Requires Updates",
        "previous_flooding_incidents": 1
    }
}

# Function to run a test case through the workflow
async def run_test_case(submission_data):
    """Run a test submission through the entire workflow"""
    # Create a new context for the test
    test_ctx = Context()
    
    # Initialize state with the submission data
    initial_state = {
        "submission_data": submission_data,
        "assessment_results": {},
        "completed_assessments": [],
        "guidelines_retrieved": {},
        "decision": {},
        "notification": {},
        "workflow_complete": False
    }
    
    await test_ctx.set("state", initial_state)
    
    # Start workflow
    logger.info(f"Starting test workflow for submission {submission_data['submission_id']}")
    
    # Process through agents sequentially
    
    # 1. Hazard Classification
    logger.info("=== Starting Hazard Classification Assessment ===")
    # Call hazard classification functions here
    
    # 2. Vulnerability Assessment
    logger.info("=== Starting Vulnerability Assessment ===")
    # Call vulnerability assessment functions here
    
    # 3. CAT Modeling
    logger.info("=== Starting CAT Modeling Assessment ===")
    # Call CAT modeling functions here
    
    # 4. Decision Making
    logger.info("=== Starting Decision Making ===")
    decision_result = await make_decision(test_ctx)
    logger.info(f"Decision result: {decision_result}")
    
    # 5. Notification
    logger.info("=== Sending Notification ===")
    # Call notification function here
    
    # Return the final state
    final_state = await test_ctx.get("state")
    return final_state

In [15]:
# Example usage
async def main():
    """Run the insurance agent system with example data"""
    
    # # Example submission data
    # example_submission = {
    #     "submission_id": "SUB20250417001",
    #     "broker_name": "ABC Insurance Brokers",
    #     "insured_name": "Acme Manufacturing Ltd",
    #     "property_details": {
    #         "address": "123 Industrial Way, Birmingham, B6 4BD",
    #         "building_type": "Manufacturing",
    #         "construction": "Steel Frame with Brick",
    #         "year_built": 1995,
    #         "area_sqm": 5000,
    #         "stories": 2,
    #         "occupancy": "Light Manufacturing - Electronics",
    #         "sprinklers": True,
    #         "alarm_system": "Grade A - 24hr Monitored"
    #     },
    #     "coverage": {
    #         "building_value": 4500000.00,
    #         "contents_value": 2000000.00,
    #         "business_interruption": 3000000.00,
    #         "deductible": 25000.00
    #     },
    #     "client_history": {
    #         "claims_count": 1,
    #         "previous_policies": 2,
    #         "risk_score": 68
    #     }
    # }

    # # Full confidence, proceed to quotation
    # example_submission = {
    #     "submission_id": "SUB20250425001",
    #     "broker_name": "SafeRisk Partners",
    #     "insured_name": "BrightWave Tech Ltd",
    #     "property_details": {
    #         "address": "200 Business Lane, San Jose, CA",
    #         "building_type": "Office",
    #         "construction": "Concrete Frame",
    #         "year_built": 2012,
    #         "area_sqm": 3000,
    #         "stories": 2,
    #         "occupancy": "Professional Offices",
    #         "sprinklers": True,
    #         "alarm_system": "Grade A - 24hr Monitored"
    #     },
    #     "coverage": {
    #         "building_value": 2800000.0,
    #         "contents_value": 1200000.0,
    #         "business_interruption": 1000000.0,
    #         "deductible": 15000.0
    #     },
    #     "client_history": {
    #         "claims_count": 0,
    #         "previous_policies": 3,
    #         "risk_score": 82
    #     }
    # }

   # Trigger human underwriting message
    example_submission = {
        "submission_id": "SUB20250425003",
        "broker_name": "EdgeCase Brokers Ltd",
        "insured_name": "Pioneer Logistics Inc",
        "property_details": {
            "address": "987 Skyline Blvd, Phoenix, AZ",
            "building_type": "Warehouse",
            "construction": "Concrete Block",
            "year_built": 1990,  #Exactly 35 years old
            "area_sqm": 4800,
            "stories": 3,        #Over 2 stories
            "occupancy": "Logistics & Storage",
            "sprinklers": True,  #Mitigates height concern
            "alarm_system": "Grade B - Monitored"
        },
        "coverage": {
            "building_value": 4100000.0,
            "contents_value": 1500000.0,
            "business_interruption": 2500000.0,
            "deductible": 20000.0
        },
        "client_history": {
            "claims_count": 1,
            "previous_policies": 2,
            "risk_score": 72
        }
    }



    
    # Initialize the system
    system = InsuranceAgentSystem()
    
    # Run the workflow
    final_state = await system.run_workflow(example_submission)
    
    # Print summary
    print(system.summarize_results(final_state))
    
    return final_state

if __name__ == "__main__":
    asyncio.run(main())

2025-04-25 08:03:41,504 - InsuranceAgentSystem - INFO - LLM and embedding models initialized
2025-04-25 08:03:41,504 - InsuranceAgentSystem - INFO - Loading existing index from storage/guidelines_index
2025-04-25 08:03:41,505 - llama_index.core.storage.kvstore.simple_kvstore - DEBUG - Loading llama_index.core.storage.kvstore.simple_kvstore from storage/guidelines_index/docstore.json.
2025-04-25 08:03:41,505 - fsspec.local - DEBUG - open file: /home/azureuser/projects/insureflow/experiments/notebooks/anna/storage/guidelines_index/docstore.json
2025-04-25 08:03:41,507 - llama_index.core.storage.kvstore.simple_kvstore - DEBUG - Loading llama_index.core.storage.kvstore.simple_kvstore from storage/guidelines_index/index_store.json.
2025-04-25 08:03:41,507 - fsspec.local - DEBUG - open file: /home/azureuser/projects/insureflow/experiments/notebooks/anna/storage/guidelines_index/index_store.json
2025-04-25 08:03:41,509 - llama_index.core.graph_stores.simple - DEBUG - Loading llama_index.core.



HUMAN REVIEW REQUIRED


2025-04-25 08:04:22,664 - httpcore.connection - DEBUG - close.started
2025-04-25 08:04:22,665 - httpcore.connection - DEBUG - close.started
2025-04-25 08:04:22,666 - InsuranceAgentSystem - INFO - Human feedback received: approve
2025-04-25 08:04:22,666 - InsuranceAgentSystem - INFO - Decision made and stored in context
2025-04-25 08:04:22,667 - httpcore.connection - DEBUG - close.complete
2025-04-25 08:04:22,669 - httpcore.connection - DEBUG - close.complete
2025-04-25 08:04:22,678 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'api-key': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a decision specialist for insurance underwriting. You analyze assessment results from multiple domains to determine whether to proceed with quotation, recommend a surveyor, or reject a submission. First, check that all required assessments (hazard, vulnerability, cat modeling) h

2025-04-25 08:04:23,300 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Transfer-Encoding', b'chunked'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'apim-request-id', b'dd758c41-7672-4e7b-a0b9-2be7278f5e44'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubDomains; preload'), (b'x-content-type-options', b'nosniff'), (b'x-ms-region', b'UK South'), (b'x-ratelimit-remaining-requests', b'2322'), (b'x-ratelimit-limit-requests', b'2330'), (b'x-ratelimit-remaining-tokens', b'205524'), (b'x-ratelimit-limit-tokens', b'233000'), (b'cmp-upstream-response-duration', b'534'), (b'x-accel-buffering', b'no'), (b'x-aml-cluster', b'hyena-spaincentral-02'), (b'x-envoy-upstream-service-time', b'513'), (b'x-ms-rai-invoked', b'true'), (b'x-request-id', b'2c69fedb-94a8-4c37-9a2d-a70241c35d00'), (b'ms-azureml-model-time', b'511'), (b'x-ms-client-request-id', b'dd758c41-7672-4e7b-a0b9-2be7278f5e44'), (b'azureml-model-session',


====== INSURANCE SUBMISSION ASSESSMENT RESULTS ======
Submission ID: SUB20250425003
Insured: Pioneer Logistics Inc

PROPERTY DETAILS:
- Address: 987 Skyline Blvd, Phoenix, AZ
- Building Type: Warehouse
- Construction: Concrete Block
- Year Built: 1990
- Sprinklers: Yes

COMPLETED ASSESSMENTS:
- hazard, vulnerability, cat_modeling

RISK ASSESSMENT SUMMARY:
- Hazard Score: 2.4/5.0
- Vulnerability Score: 1.8/5.0
- CAT Risk Score: 2.0/5.0
- Composite Score: 2.1/5.0

DECISION: RECOMMEND_SURVEYOR
Reason: Risk profile requires additional assessment
Confidence: 80%

HUMAN FEEDBACK:
Status: completed
Comment: approve



2025-04-25 08:04:24,251 - httpcore.http11 - DEBUG - receive_response_body.complete
2025-04-25 08:04:24,256 - httpcore.http11 - DEBUG - response_closed.started
2025-04-25 08:04:24,258 - httpcore.http11 - DEBUG - response_closed.complete
2025-04-25 08:04:24,263 - InsuranceAgentSystem - INFO - Starting notification preparation
2025-04-25 08:04:24,268 - InsuranceAgentSystem - INFO - Email Body:

Subject: Submission SUB20250425003 Requires Surveyor Assessment

Dear Distribution Team,

The submission for Pioneer Logistics Inc (ID: SUB20250425003) has been reviewed and requires a surveyor assessment before proceeding.

Risk Assessment Summary:
- Hazard Score: 2.4/5.0
- Vulnerability Score: 1.8/5.0
- CAT Risk Score: 2.0/5.0

Reason for surveyor recommendation: Risk profile requires additional assessment
Confidence: 80%

Please arrange for a risk assessment survey.

Regards,
Underwriting AI Assistant

2025-04-25 08:04:24,269 - InsuranceAgentSystem - INFO - Email notification prepared for distri